In [ ]:
#Ex1
import os
import numpy as np
import torch
from PIL import Image

# Define the tumor types and splits
tumor_types = ['glioma_tumor', 'no_tumor', 'pituitary_tumor', 'meningioma_tumor']
splits = ['Testing', 'Training']

# Initialize empty lists for train and test sets
train_set, test_set = [], []

# Define a function to convert the label to a one-hot encoded tensor
def get_label(tumor_type):
    labels = {
        'glioma_tumor': [1, 0, 0, 0],
        'no_tumor': [0, 0, 0, 1],
        'pituitary_tumor': [0, 0, 1, 0],
        'meningioma_tumor': [0, 1, 0, 0],
    }
    return torch.tensor(labels[tumor_type], dtype=torch.float32)

# Process each tumor type and split
for tumor in tumor_types:
    for split in splits:
        dir_path = f'brain_mri/{split}/{tumor}'
        for img_name in os.listdir(dir_path):
            img_path = os.path.join(dir_path, img_name)
            img_pil = Image.open(img_path).resize((128, 128))
            img_np = np.array(img_pil)
            img_np = np.rollaxis(img_np, -1, 0)
            img_tensor = torch.tensor(img_np, dtype=torch.float32)
            label_tensor = get_label(tumor)
            # Standardize the image
            img_tensor = (img_tensor - 63.64) / 54.89
            # Append to the respective dataset
            if split == 'Training':
                train_set.append((img_tensor, label_tensor))
            else:
                test_set.append((img_tensor, label_tensor))

# Optionally, you can convert the lists to tuples if needed
# train_set, test_set = tuple(train_set), tuple(test_set)

# Example to verify the shape and type of the first training sample
print(train_set[0][0].shape, train_set[0][0].dtype)  # Should print: torch.Size([3, 128, 128]) torch.float32
print(train_set[0]A[1])  # Should print the one-hot encoded label tensor

# Save the datasets if needed
torch.save(train_set, 'train_set.pt')
torch.save(test_set, 'test_set.pt')


In [2]:
#Ex2
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvolutionalNN(nn.Module):
    def __init__(self):
        super(ConvolutionalNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        self.adaptive_pool = nn.AdaptiveAvgPool2d(output_size=4)
        
        self.fc1 = nn.Linear(in_features=64 * 4 * 4, out_features=16)
        self.fc2 = nn.Linear(in_features=16, out_features=4)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.bn1(x)
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.bn2(x)
        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.bn3(x)
        x = self.adaptive_pool(x)
        
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        
        return x
    
    def get_number_of_parameters(self):
        return sum(p.numel() for p in self.parameters())

# Instantiate the model and print the number of parameters
model = ConvolutionalNN()
print("Number of parameters:", model.get_number_of_parameters())

# Example to verify the forward pass
input_tensor = torch.randn(1, 3, 128, 128)
output_tensor = model(input_tensor)
print("Output shape:", output_tensor.shape)  # Should print: torch.Size([1, 4])


Number of parameters: 52612
Output shape: torch.Size([1, 4])


In [3]:
#Ex3
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = ConvolutionalNN().to(device)

print('model number of parameters:', model.get_number_of_parameters())


model number of parameters: 52612


In [4]:
#Ex4
import torch
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

# Assuming train_set and test_set are already defined as lists of tuples (image, label)
# If not, you can load them from previously saved files
# train_set = torch.load('train_set.pt')
# test_set = torch.load('test_set.pt')

# Define the DataLoaders
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

# Define the loss function
loss_function = CrossEntropyLoss()

# Define the optimizer
optimizer = Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)

# Print to verify
print('Train loader batch size:', train_loader.batch_size)
print('Test loader batch size:', test_loader.batch_size)
print('Optimizer:', optimizer)
print('Loss function:', loss_function)


Train loader batch size: 32
Test loader batch size: 1
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0005
    maximize: False
    weight_decay: 1e-05
)
Loss function: CrossEntropyLoss()


In [ ]:
#Ex5
epochs = 60
n_samples_train = len(train_loader)
n_samples_test = len(test_loader)
training_loss_per_epoch, test_loss_per_epoch, test_accuracy_per_epoch = [], [], []
best_accuracy = - torch.inf
for epoch in range(epochs):
    training_loss = 0
    for data, labels in train_loader:
        predict = model(data.to(device))
        loss = loss_function(labels.to(device), predict)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_loss += (loss.item() / n_samples_train)
    test_loss, correct, total = (0, 0, 0)
    for data, label in test_loader:
        predict = model(data.to(device))
        loss = loss_function(label.to(device), predict)
        test_loss += (loss.item() / n_samples_test)
        if torch.argmax(predict) == torch.argmax(label):
            correct += 1
        total += 1
    accuracy = correct/total*100
    if accuracy > best_accuracy:
        best_accuracy = accuracy
    training_loss_per_epoch.append(training_loss)
    test_loss_per_epoch.append(test_loss)
    test_accuracy_per_epoch.append(accuracy)
    print('''Epoch {}: training loss: {:.3f}, test loss: {:.3f}, test accuracy: {:.2f}%
          '''.format(epoch+1, training_loss, test_loss, accuracy))
print(f'\n Best test accuracy: {best_accuracy}')


Epoch 1: training loss: 1.229, test loss: 1.506, test accuracy: 24.62%
          
Epoch 2: training loss: 1.035, test loss: 1.492, test accuracy: 25.63%
          
Epoch 3: training loss: 0.977, test loss: 1.482, test accuracy: 27.66%
          
Epoch 4: training loss: 0.954, test loss: 1.444, test accuracy: 31.98%
          
Epoch 5: training loss: 0.927, test loss: 1.440, test accuracy: 31.47%
          
Epoch 6: training loss: 0.920, test loss: 1.411, test accuracy: 33.50%
          
Epoch 7: training loss: 0.911, test loss: 1.426, test accuracy: 32.49%
          
Epoch 8: training loss: 0.896, test loss: 1.390, test accuracy: 36.80%
          
Epoch 9: training loss: 0.884, test loss: 1.367, test accuracy: 38.07%
          
Epoch 10: training loss: 0.881, test loss: 1.359, test accuracy: 38.83%
          
Epoch 11: training loss: 0.876, test loss: 1.372, test accuracy: 38.32%
          
Epoch 12: training loss: 0.867, test loss: 1.372, test accuracy: 36.55%
          
Epoch 13: tra

In [ ]:
#Ex6
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(12,5))
ax[0].plot(test_loss_per_epoch, c='r')
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Loss', fontsize=16)
ax[1].plot(test_accuracy_per_epoch, c='g')
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Accuracy %', fontsize=16)
